In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy

import torch.nn.init as init

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())


PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Convert to tensor and scale to [0, 1]
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('/home/c/cl237/', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='/home/c/cl237/', train=False, download=True, transform=ts)

In [3]:
from torch.utils.data import Subset
train_subset_size = int(0.1 * len(mnist_trainset))
test_subset_size = int(0.0999 * len(mnist_testset))
train_indices = list(range(len(mnist_trainset)))
test_indices = list(range(len(mnist_testset)))
torch.manual_seed(10)

seed_value = 10
np.random.seed(seed_value)

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

train_subset_indices = train_indices[:train_subset_size]
test_subset_indices = test_indices[:test_subset_size]

mnist_trainset = Subset(mnist_trainset, train_subset_indices)
mnist_testset = Subset(mnist_testset, test_subset_indices)

In [4]:
x_d0 = mnist_trainset[0][0].size()[0]
x_d1 = mnist_trainset[0][0].size()[1]
x_d2 = mnist_trainset[0][0].size()[2]
# ([1, 28, 28])
N = x_d3 = len(mnist_trainset)
K = 10
x_train = torch.empty((N,x_d0*x_d1*x_d2), device=device)
# (60000, 28*28)

y_train = torch.empty(N, dtype=torch.long)


for i in range(N):
     x_train[i,:] = torch.reshape(mnist_trainset[i][0], (1, x_d0*x_d1*x_d2))
     y_train[i] = mnist_trainset[i][1]
x_train = torch.t(x_train)
y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

# Manipulate test set
N_test = x_d3_test = len(mnist_testset)
x_test = torch.empty((N_test,x_d0*x_d1*x_d2), device=device)
y_test = torch.empty(N_test, dtype=torch.long)
for i in range(N_test):
     x_test[i,:] = torch.reshape(mnist_testset[i][0], (1, x_d0*x_d1*x_d2))
     y_test[i] = mnist_testset[i][1]
x_test = torch.t(x_test)
y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [5]:
#### Reshape X to X-bar
x_trainTensor = torch.reshape(x_train, (x_d1, x_d2, x_d0,-1))
x_testTensor  = torch.reshape(x_test, (x_d1, x_d2, x_d0,-1))
print(x_trainTensor.shape)
print(x_testTensor.shape)

torch.Size([28, 28, 1, 6000])
torch.Size([28, 28, 1, 999])


In [6]:
def inputX_CNN(x_Tensor, filter_size, stride):
    H, W, C, n = x_Tensor.size()
    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
    Hprime = Hprime.to(torch.int)
    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
    Wprime = Wprime.to(torch.int)
    Xtranform = torch.zeros((Hprime * Wprime, filter_size * filter_size * C, n), device=device)
    for i in range(n):
      Data =  x_Tensor[:,:,:,i]
      for hh in range(Hprime):
        for ww in range(Wprime):
          #  print(range(ww * stride, ww * stride + filter_size))
            DataTemp = Data[range(hh * stride, hh * stride + filter_size), :,:]
            DataTemp = DataTemp[:, range(ww * stride, ww * stride + filter_size), :]
            Xtranform[Hprime * hh + ww, :, i] = torch.reshape(DataTemp, (1, filter_size * filter_size * C))
    return Xtranform

In [7]:
### from X-bar to X-bar-bar
filter_size=3
stride = 2
x_trainTS = inputX_CNN(x_trainTensor,filter_size,stride)
x_testTS  = inputX_CNN(x_testTensor,filter_size,stride)

In [8]:

def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)  
    d,N = V.size()
    I = torch.eye(N, device=device)
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+(alpha+tau)*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix)
    return Wstar

def updateWb_CNNorg(U, V, W, alpha, rho):
    d,N = V.size()
    I = torch.eye(N, device=device)
    #_, col_U = U.size()
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+alpha*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W)
    
    return Wstar

def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [ ]:
niter = 2000
rank = 220
tau = 0.2
gamma = 0.5
alpha = 1
rho = 0.5

print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)

S = 32 ### number of filters 2^5
H, W, C, n = x_trainTensor.size()   # n is the same thing as N
Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
Hprime = Hprime.to(torch.int)
Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
Wprime = Wprime.to(torch.int)



for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d1 =  Hprime * Wprime * S
    d2 =  1024
    d3 =  1024
    d4 =  10


    W1 = 0.01*torch.randn(filter_size * filter_size * C, S, device=device)  
    b1 = 0*torch.ones(d1, 1, device=device) 


    W2 = init.xavier_uniform_(torch.empty(d2, d1, device=device))
    W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)) 
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = init.xavier_uniform_(torch.empty(d3, d2, device=device))
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) 
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = init.xavier_uniform_(torch.empty(d4, d3, device=device))
    b4 = 0*torch.ones(d4, 1, device=device)

    x_trainTS1 = torch.reshape(x_trainTS, (-1,n))
    x_trainTS1 = torch.t(x_trainTS1)
    x_trainTS1 = torch.reshape(x_trainTS1, (-1, filter_size * filter_size * C))   
    U1prime = torch.matmul(x_trainTS1, W1)
    U1prime = torch.reshape(U1prime, (n,-1))
    U1 = torch.t(U1prime)

    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = U4

    x_testTS1 = torch.reshape(x_testTS, (-1, N_test))
    x_testTS1 = torch.t(x_testTS1)
    x_testTS1 = torch.reshape(x_testTS1, (-1, filter_size * filter_size * C))   


    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

  # update for last layer
        # update V4
        V4 = (y_one_hot + gamma*U4 + alpha*V4)/(1 + gamma + alpha)

        # update U4
        U4 = (gamma*V4 + rho*(torch.mm(W4,V3) + b4.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W4, b4 = updateWb_org(U4,V3,W4,b4,alpha,rho)


  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


 # update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        XprimeW = torch.reshape(torch.matmul(x_trainTS1, W1), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        # update U1
        U1 = relu_prox(V1,(rho*XprimeWtranspose + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1 = updateWb_CNNorg(U1,x_trainTS1,W1,alpha,rho)






  # prediction for trainning data
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        a1_train = nn.ReLU()(XprimeWtranspose)
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        pred = torch.argmax(torch.addmm(b4.repeat(1, N), W4, a3_train), dim=0)

  #Prediction for test data
        XprimeWtest = torch.reshape(torch.matmul(x_testTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (N_test,-1))
        XprimeWtesttranspose = torch.t(XprimeWtest)
        a1_test = nn.ReLU()(XprimeWtesttranspose) 
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        pred_test = torch.argmax(torch.addmm(b4.repeat(1, N_test), W4, a3_test), dim=0)


    #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V4,y_one_hot,2),2).cpu().numpy()

        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(XprimeWtranspose,U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,U4,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    total_variabels=Sum_of_variables_factors2+Sum_of_variables_factors3

    layer2_CR = Sum_of_variables_factors2/(d1*d2).item()
    layer3_CR = Sum_of_variables_factors3/(d2*d3)
    Compressedlayers_CR = total_variabels/(d1*d2+d2*d3).item()
    Compressedlayers_CR2 = (total_variabels+d3*d4)/(d1*d2+d2*d3+d3*d4).item()



    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(layer2_CR,layer3_CR,Compressedlayers_CR,Compressedlayers_CR2)



rank= 220 tau= 0.2 gamma= 0.5 rho= 0.5 alpha 1
Train on 6000 samples, validate on 999 samples
Repeatition 1 Epoch 1 / 2000 
 - time: 5.700935363769531 - sq_loss: 542.602294921875 - tot_loss: 790.8065661418368 - acc: 0.1975 - val_acc: 0.2002002002002002
Repeatition 1 Epoch 2 / 2000 
 - time: 5.607478857040405 - sq_loss: 241.1565399169922 - tot_loss: 445.6043138843961 - acc: 0.34833333333333333 - val_acc: 0.36236236236236236
Repeatition 1 Epoch 3 / 2000 
 - time: 5.630671262741089 - sq_loss: 121.41243743896484 - tot_loss: 263.2274810664356 - acc: 0.4698333333333333 - val_acc: 0.46646646646646645
Repeatition 1 Epoch 4 / 2000 
 - time: 5.623109817504883 - sq_loss: 63.823333740234375 - tot_loss: 159.34886711202563 - acc: 0.563 - val_acc: 0.5495495495495496
Repeatition 1 Epoch 5 / 2000 
 - time: 5.617477893829346 - sq_loss: 34.075233459472656 - tot_loss: 99.21060183942318 - acc: 0.617 - val_acc: 0.6116116116116116
Repeatition 1 Epoch 6 / 2000 
 - time: 5.609032392501831 - sq_loss: 18.3171043

Repeatition 1 Epoch 49 / 2000 
 - time: 5.5775628089904785 - sq_loss: 0.00021518339053727686 - tot_loss: 0.5280458028253634 - acc: 0.8958333333333334 - val_acc: 0.8958958958958959
Repeatition 1 Epoch 50 / 2000 
 - time: 5.599209308624268 - sq_loss: 0.000204500844120048 - tot_loss: 0.5147330951964251 - acc: 0.8965 - val_acc: 0.8968968968968969
Repeatition 1 Epoch 51 / 2000 
 - time: 5.606887102127075 - sq_loss: 0.00019480638729874045 - tot_loss: 0.5020934230909916 - acc: 0.8978333333333334 - val_acc: 0.8988988988988988
Repeatition 1 Epoch 52 / 2000 
 - time: 5.577648639678955 - sq_loss: 0.00018599619215819985 - tot_loss: 0.4900798016489716 - acc: 0.8986666666666666 - val_acc: 0.8988988988988988
Repeatition 1 Epoch 53 / 2000 
 - time: 5.588446140289307 - sq_loss: 0.00017799163470044732 - tot_loss: 0.4786480235750787 - acc: 0.8995 - val_acc: 0.9009009009009009
Repeatition 1 Epoch 54 / 2000 
 - time: 5.591747760772705 - sq_loss: 0.00017065112479031086 - tot_loss: 0.4677568523446098 - acc: 

Repeatition 1 Epoch 97 / 2000 
 - time: 5.579739332199097 - sq_loss: 6.542485789395869e-05 - tot_loss: 0.24149917046306654 - acc: 0.925 - val_acc: 0.9219219219219219
Repeatition 1 Epoch 98 / 2000 
 - time: 5.563257932662964 - sq_loss: 6.449878856074065e-05 - tot_loss: 0.2389148280140944 - acc: 0.9261666666666667 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 99 / 2000 
 - time: 5.579142808914185 - sq_loss: 6.360394036164507e-05 - tot_loss: 0.23638843816443114 - acc: 0.927 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 100 / 2000 
 - time: 5.589879512786865 - sq_loss: 6.272819882724434e-05 - tot_loss: 0.23391880165727344 - acc: 0.9275 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 101 / 2000 
 - time: 5.605987548828125 - sq_loss: 6.188085535541177e-05 - tot_loss: 0.23150380697334186 - acc: 0.9281666666666667 - val_acc: 0.9209209209209209
Repeatition 1 Epoch 102 / 2000 
 - time: 5.586994886398315 - sq_loss: 6.10474671702832e-05 - tot_loss: 0.22914095783780794 - acc: 0.9281666666

Repeatition 1 Epoch 144 / 2000 
 - time: 5.612772703170776 - sq_loss: 3.8265137845883146e-05 - tot_loss: 0.1614518949885678 - acc: 0.9406666666666667 - val_acc: 0.9309309309309309
Repeatition 1 Epoch 145 / 2000 
 - time: 5.602189540863037 - sq_loss: 3.791560448007658e-05 - tot_loss: 0.16034405712707667 - acc: 0.9408333333333333 - val_acc: 0.9309309309309309
Repeatition 1 Epoch 146 / 2000 
 - time: 5.600365400314331 - sq_loss: 3.7571287975879386e-05 - tot_loss: 0.15925211070352815 - acc: 0.941 - val_acc: 0.9309309309309309
Repeatition 1 Epoch 147 / 2000 
 - time: 5.579147815704346 - sq_loss: 3.722726978594437e-05 - tot_loss: 0.15817559634888312 - acc: 0.941 - val_acc: 0.9309309309309309
Repeatition 1 Epoch 148 / 2000 
 - time: 5.6029462814331055 - sq_loss: 3.6891960917273536e-05 - tot_loss: 0.15711423117681989 - acc: 0.9413333333333334 - val_acc: 0.9309309309309309
Repeatition 1 Epoch 149 / 2000 
 - time: 5.572471857070923 - sq_loss: 3.656136686913669e-05 - tot_loss: 0.15606759699294342

Repeatition 1 Epoch 191 / 2000 
 - time: 5.595446825027466 - sq_loss: 2.6394771339255385e-05 - tot_loss: 0.1223034535007173 - acc: 0.9505 - val_acc: 0.9429429429429429
Repeatition 1 Epoch 192 / 2000 
 - time: 5.580636739730835 - sq_loss: 2.6217703634756617e-05 - tot_loss: 0.12168527092253499 - acc: 0.9505 - val_acc: 0.9429429429429429
Repeatition 1 Epoch 193 / 2000 
 - time: 5.56907057762146 - sq_loss: 2.6043275283882394e-05 - tot_loss: 0.12107344423930044 - acc: 0.9511666666666667 - val_acc: 0.9429429429429429
Repeatition 1 Epoch 194 / 2000 
 - time: 5.554444789886475 - sq_loss: 2.5870058379950933e-05 - tot_loss: 0.12046796381291643 - acc: 0.9513333333333334 - val_acc: 0.9429429429429429
Repeatition 1 Epoch 195 / 2000 
 - time: 5.593254804611206 - sq_loss: 2.5697232558741234e-05 - tot_loss: 0.1198688183005288 - acc: 0.9513333333333334 - val_acc: 0.9429429429429429
Repeatition 1 Epoch 196 / 2000 
 - time: 5.596506595611572 - sq_loss: 2.552748264861293e-05 - tot_loss: 0.1192759472171019

...

Repeatition 1 Epoch 1744 / 2000 
 - time: 6.184159994125366 - sq_loss: 1.7415113688912243e-06 - tot_loss: 0.017455251674164174 - acc: 0.9921666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1745 / 2000 
 - time: 6.188110828399658 - sq_loss: 1.741147571010515e-06 - tot_loss: 0.01744627812904582 - acc: 0.9921666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1746 / 2000 
 - time: 6.235360145568848 - sq_loss: 1.740489324220107e-06 - tot_loss: 0.01743731781257338 - acc: 0.9921666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1747 / 2000 
 - time: 6.204703092575073 - sq_loss: 1.7395642544215661e-06 - tot_loss: 0.017428368860805676 - acc: 0.9921666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1748 / 2000 
 - time: 6.238610744476318 - sq_loss: 1.7380865529048606e-06 - tot_loss: 0.01741942747337362 - acc: 0.9921666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1749 / 2000 
 - time: 6.221045970916748 - sq_loss: 1.7364302493660944e

Repeatition 1 Epoch 1790 / 2000 
 - time: 6.169431447982788 - sq_loss: 1.6834117104735924e-06 - tot_loss: 0.017052073063359787 - acc: 0.9926666666666667 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1791 / 2000 
 - time: 6.19196081161499 - sq_loss: 1.6824825479488936e-06 - tot_loss: 0.017043516587114026 - acc: 0.9926666666666667 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1792 / 2000 
 - time: 6.184694290161133 - sq_loss: 1.6815477010823088e-06 - tot_loss: 0.017034952638596224 - acc: 0.9926666666666667 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1793 / 2000 
 - time: 6.207351446151733 - sq_loss: 1.6802662230475107e-06 - tot_loss: 0.01702638755918997 - acc: 0.9926666666666667 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1794 / 2000 
 - time: 6.245701313018799 - sq_loss: 1.6791175312391715e-06 - tot_loss: 0.01701782987588558 - acc: 0.9926666666666667 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1795 / 2000 
 - time: 6.187914609909058 - sq_loss: 1.677852424109005

Repeatition 1 Epoch 1837 / 2000 
 - time: 6.155400514602661 - sq_loss: 1.6273883147732704e-06 - tot_loss: 0.016658766697923967 - acc: 0.993 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1838 / 2000 
 - time: 6.194267272949219 - sq_loss: 1.626405946808518e-06 - tot_loss: 0.016650591157076632 - acc: 0.993 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1839 / 2000 
 - time: 6.178842782974243 - sq_loss: 1.6255610262305709e-06 - tot_loss: 0.016642425439067667 - acc: 0.993 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1840 / 2000 
 - time: 6.176433086395264 - sq_loss: 1.6239945352936047e-06 - tot_loss: 0.0166342672458768 - acc: 0.993 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1841 / 2000 
 - time: 6.199743032455444 - sq_loss: 1.6224881846937933e-06 - tot_loss: 0.016626114888299525 - acc: 0.993 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1842 / 2000 
 - time: 6.22678279876709 - sq_loss: 1.621945557417348e-06 - tot_loss: 0.016617970486004194 - acc: 0.993 - val_acc: 0.97

Repeatition 1 Epoch 1884 / 2000 
 - time: 6.231486558914185 - sq_loss: 1.573386157360801e-06 - tot_loss: 0.016283252853168048 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1885 / 2000 
 - time: 6.198997735977173 - sq_loss: 1.5722182524768868e-06 - tot_loss: 0.01627544547013713 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1886 / 2000 
 - time: 6.24102520942688 - sq_loss: 1.5708238834122312e-06 - tot_loss: 0.016267647126664998 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1887 / 2000 
 - time: 6.234940052032471 - sq_loss: 1.5693713066866621e-06 - tot_loss: 0.016259853519568424 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1888 / 2000 
 - time: 6.16473388671875 - sq_loss: 1.5680776641602279e-06 - tot_loss: 0.016252069604001915 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1889 / 2000 
 - time: 6.202721834182739 - sq_loss: 1.567050958328764e

Repeatition 1 Epoch 1929 / 2000 
 - time: 6.205889940261841 - sq_loss: 1.5238734931699582e-06 - tot_loss: 0.01593886339890105 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1930 / 2000 
 - time: 6.168067693710327 - sq_loss: 1.522910224593943e-06 - tot_loss: 0.015931373538751358 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1931 / 2000 
 - time: 6.160979986190796 - sq_loss: 1.5225232345983386e-06 - tot_loss: 0.015923880811078563 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1932 / 2000 
 - time: 6.182160377502441 - sq_loss: 1.5221121429931372e-06 - tot_loss: 0.01591640353535695 - acc: 0.9935 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1933 / 2000 
 - time: 6.194714546203613 - sq_loss: 1.5205001773210824e-06 - tot_loss: 0.015908944244642953 - acc: 0.9935 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1934 / 2000 
 - time: 6.234293460845947 - sq_loss: 1.5195147398117115e-06 - tot_loss: 0.0159